# 3.3: Split-apply-combine in pandas

Now that we are (somewhat) comfortable with transformation of data between wide and long, we can get into another very powerful pandas feature known as split-apply-combine.

---

## Dataset

The three files are in your ```../assets/datasets/``` directory. They are:

- ```mach_data.csv``` which contains the wide data.
- ```mach_long.csv``` which contains the already long data.
- ```mach_codebook.csv``` which contains the information about the survey data.

---

## Packages

Loaded same as ever.

In [1]:
!pwd

/Users/davidswan/Desktop/GA WORK/curriculum/week-02/3.3-lesson/code


In [2]:
# data modules
import numpy as np
import scipy.stats as stats
import pandas as pd

# plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# make sure charts appear in the notebook:
%matplotlib inline

---

## A: Load the already widened data

I have already widened the data for you here in the interest of time. You can see how I get started on the bonus 2 question below as well.

If you would like to do the melting part yourself for practice, be my guest! The more practice the better. If doing it yourself you would instead load the ```mach_data.csv``` file again.

In [15]:
mach_long = pd.read_csv('/Users/davidswan/Downloads/mach_long.csv')
mach_long

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
1,33,2,2,Q1,2
2,21,1,3,Q1,3
3,17,1,4,Q1,4
4,22,1,5,Q1,4
5,21,1,6,Q1,5
6,55,1,7,Q1,4
7,40,1,8,Q1,1
8,30,2,9,Q1,3
9,65,1,10,Q1,2


In [4]:
print mach_long.variable.unique()

['Q1' 'Q2' 'Q3' 'Q4' 'Q5' 'Q6' 'Q7' 'Q8' 'Q9' 'Q10' 'Q11' 'Q12' 'Q13' 'Q14'
 'Q15' 'Q16' 'Q17' 'Q18' 'Q19' 'Q20' 'score' 'seconds_elapsed']


In [5]:
mach_long = mach_long[mach_long.variable != 'score']

seconds_data = mach_long.ix[mach_long.variable == 'seconds_elapsed', ['subject_id','variable','value']]

seconds_data['seconds_per_q'] = seconds_data.value / 20.

mach_long = mach_long[mach_long.variable != 'seconds_elapsed']

seconds_data.head()

,subject_id,variable,value,seconds_per_q
255906,1,seconds_elapsed,177,8.85
255907,2,seconds_elapsed,107,5.35
255908,3,seconds_elapsed,323,16.15
255909,4,seconds_elapsed,136,6.80
255910,5,seconds_elapsed,142,7.10


### A.2 : Sorting by multiple columns with custom sort for questions

We are going to use ```pd.Categorical``` to custom-sort the questions after subject

see:  http://stackoverflow.com/questions/13838405/custom-sorting-in-pandas-dataframe

Look stuff up online!!

In [6]:
categorical_questions = pd.Categorical(mach_long.variable,
                                       ['Q1','Q2','Q3','Q4',
                                        'Q5','Q6','Q7','Q8',
                                        'Q9','Q10','Q11','Q12',
                                        'Q13','Q14','Q15','Q16',
                                        'Q17','Q18','Q19','Q20'])

mach_long['variable'] = categorical_questions

mach_long.sort_values(['subject_id','variable'], inplace=True)

mach_long.head()

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
12186,24,1,1,Q2,4
24372,24,1,1,Q3,2
36558,24,1,1,Q4,2
48744,24,1,1,Q5,4


### A.3: Adding the time by question by subject_id as a new value

Merge the seconds data and the long data together. This is a preview to future lessons!

In [7]:
mach_secs = mach_long.merge(seconds_data[['subject_id', 'seconds_per_q']], on='subject_id', sort=False)

mach_secs.head()

,age,gender,subject_id,variable,value,seconds_per_q
0,24,1,1,Q1,4,8.85
1,24,1,1,Q2,4,8.85
2,24,1,1,Q3,2,8.85
3,24,1,1,Q4,2,8.85
4,24,1,1,Q5,4,8.85


### A.4 Doing a cumulative sum of the seconds

Do a cumulative sum of the seconds by subject.

This is an example of the split-apply-combine pattern. We will do more as well.

#### A.4.1: Split the data into groups by subject_id

In [11]:
#mach_long
pd.pivot_table(mach_secs, index = "subject_id")
mach_long

,age,gender,subject_id,variable,value
0,24,1,1,Q1,4
12186,24,1,1,Q2,4
24372,24,1,1,Q3,2
36558,24,1,1,Q4,2
48744,24,1,1,Q5,4
60930,24,1,1,Q6,2
73116,24,1,1,Q7,3
85302,24,1,1,Q8,5
97488,24,1,1,Q9,3
109674,24,1,1,Q10,4


In [14]:
mach_secs_wide = pd.pivot_table(mach_secs,
                                values=['value','seconds_elapsed'],
                                index=['subject_id','age','gender'],
                                columns=['variable'])
mach_secs_wide

value                                                    \
variable                 Q1 Q10 Q11 Q12 Q13 Q14 Q15 Q16 Q17 Q18 Q19 Q2 Q20 Q3   
subject_id age gender                                                           
1          24  1          4   4   2   5   2   1   4   4   2   0   5  4   2  2   
2          33  2          2   4   2   4   2   2   3   2   2   4   5  2   2  4   
3          21  1          3   3   2   4   4   3   3   4   2   4   5  4   1  2   
4          17  1          4   3   1   5   3   2   4   2   2   3   4  4   2  3   
5          22  1          4   3   1   2   4   4   5   4   1   4   5  5   4  2   
6          21  1          5   2   2   4   4   2   4   2   2   4   4  5   5  5   
7          55  1          4   2   1   4   4   2   4   1   2   5   5  4   3  3   
8          40  1          1   4   2   3   1   4   3   4   3   3   4  3   1  5   
9          30  2          3   4   1   5   4   2   1   2   2   4   5  3   2  5   
10         65  1          2   5   2   4   3   4   2   2   2   3   4  1   4  2   
11         21  1          3   4   3   4   2   2   4   4   4   2   5  2   1  4   
12         23  1          5   3   1   4   4   2   3   2   3   3   4  3   4  2   
13         17  1          4   4   3   5   2   4   2   2   2   4   4  2   2  3   
14         24  1          3   4   2   4   3   3   4   2   2   3   5  4   2  4   
15         20  1          4   2   2   2   3   3   4   2   2   4   4  4   2  3   
16         20  1          3   3   1   4   5   2   5   4   2   4   5  4   5  3   
17         27  1          4   3   1   5   4   1   2   3   1   5   3  3   3  2   
18         48  2          2   4   3   3   2   4   4   2   3   4   2  4   2  4   
19         54  1          4   2   2   1   2   2   3   1   1   4   5  5   1  3   
20         17  1          3   4   4   3   4   2   3   2   3   3   4  4   2  4   
21         20  1          2   4   2   3   2   2   2   2   4   4   4  2   1  2   
22         24  1          5   2   2   5   2   3   2   2   2   4   4  4   4  1   
23         20  2          4   4   1   4   3   2   2   2   2   3   3  2   3  3   
24         19  1          5   2   1   1   3   2   3   1   2   4   4  3   4  2   
25         26  2          5   3   1   3   1   1   3   3   2   3   5  5   1  4   
26         32  1          2   2   2   4   2   2   4   2   2   4   5  4   2  2   
27         56  1          2   3   2   4   3   3   3   2   2   4   4  3   2  3   
28         31  1          2   2   2   4   4   3   4   2   4   5   5  4   2  1   
29         29  2          4   4   3   4   2   2   2   4   2   4   3  2   1  2   
30         30  2          2   3   1   4   4   2   2   2   2   5   5  4   4  2   
...                     ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  .. ..  .. ..   
12157      32  1          2   3   1   3   4   2   4   4   4   0   5  3   4  4   
12158      70  1          2   2   3   4   3   2   2   4   2   3   2  2   2  3   
12159      29  2          1   5   2   3   1   2   1   2   2   2   5  2   1  4   
12160      23  1          5   1   1   1   1   1   1   1   1   5   5  5   5  1   
12161      19  1          4   2   1   5   4   1   3   2   4   5   5  3   3  2   
12162      27  1          5   2   1   4   4   2   4   4   2   4   5  4   2  2   
12163      50  2          2   3   2   2   1   2   2   3   3   2   4  1   1  2   
12164      23  1          2   4   2   2   2   2   2   4   3   4   4  4   3  2   
12165      23  1          4   1   1   5   5   1   4   5   2   5   5  4   1  2   
12166      59  1          3   4   2   4   4   3   2   2   4   4   5  4   3  2   
12167      17  1          3   2   3   3   5   2   4   3   3   4   4  3   2  2   
12168      24  1          5   1   1   5   5   1   5   1   1   5   5  5   5  1   
12169      18  1          4   1   2   5   4   2   3   2   2   2   5  5   2  2   
12170      22  2          2   3   2   3   3   2   2   2   2   3   5  3   3  3   
12171      39  1          4   4   3   4   4   2   4   2   2   4   4  4   3  3   
12172      39  1          4   4   2   4   4   2   4  

#### A.4.2: Apply the cumulative sum function, iterating through the groups

the ```.transform()``` function on group objects requires a function as the argument.